In [ ]:
class LR0Parser:
    def __init__(self, grammar):
        self.grammar = grammar
        self.states = []
        self.goto_table = {}
        self.action_table = {}

    def closure(self, items):
        closure_items = set(items)

        while True:
            changed = False
            for item in closure_items.copy():
                dot_index = item[1].index('.')
                if dot_index < len(item[1]) - 1 and item[1][dot_index + 1] in self.grammar:
                    next_symbol = item[1][dot_index + 1]
                    for production in self.grammar[next_symbol]:
                        new_item = (next_symbol, '.' + production)
                        if new_item not in closure_items:
                            closure_items.add(new_item)
                            changed = True

            if not changed:
                break

        return closure_items

    def goto(self, items, symbol):
        goto_items = set()

        for item in items:
            dot_index = item[1].index('.')
            if dot_index < len(item[1]) - 1 and item[1][dot_index + 1] == symbol:
                new_item = (item[0], item[1][:dot_index] + symbol + '.' + item[1][dot_index + 2:])
                goto_items.add(new_item)

        return self.closure(goto_items)

    def build_states(self):
        initial_state = self.closure({('S\'', '.S')})
        self.states.append(initial_state)

        i = 0
        while i < len(self.states):
            current_state = self.states[i]

            for symbol in self.get_symbols():
                goto_result = self.goto(current_state, symbol)
                if goto_result and goto_result not in self.states:
                    self.states.append(goto_result)

                if goto_result:
                    self.goto_table[(i, symbol)] = self.states.index(goto_result)

            i += 1

    def build_parsing_table(self):
        self.build_states()

        for i, state in enumerate(self.states):
            for item in state:
                dot_index = item[1].index('.')
                if dot_index < len(item[1]) - 1:
                    next_symbol = item[1][dot_index + 1]
                    if next_symbol in self.grammar:
                        goto_state = self.goto(state, next_symbol)
                        self.action_table[(i, next_symbol)] = ('S', self.states.index(goto_state))

                elif item[0] != 'S\'':
                    production_index = self.grammar[item[0]].index(item[1][:dot_index])
                    for symbol in self.get_follow(item[0]):
                        self.action_table[(i, symbol)] = ('R', (item[0], production_index))

    def get_symbols(self):
        symbols = set()
        for state in self.states:
            for item in state:
                dot_index = item[1].index('.')
                if dot_index < len(item[1]) - 1:
                    symbols.add(item[1][dot_index + 1])
        return symbols

    def get_follow(self, non_terminal):
        follow_set = set()
        for symbol in self.get_symbols():
            if non_terminal in self.grammar and symbol in self.grammar[non_terminal]:
                follow_set.add(symbol)
        return follow_set


# Example Grammar
grammar = {
    'S\'': ['S'],
    'S': ['C C'],
    'C': ['c C', 'd']
}

lr0_parser = LR0Parser(grammar)
lr0_parser.build_parsing_table()

# Display LR(0) items for each state
for i, state in enumerate(lr0_parser.states):
    print(f"I{i}: {state}")

# Display Goto Table
print("\nGoto Table:")
for key, value in lr0_parser.goto_table.items():
    print(f"Goto(I{key[0]}, {key[1]}) = I{value}")

# Display Action Table
print("\nAction Table:")
for key, value in lr0_parser.action_table.items():
    print(f"Action(I{key[0]}, {key[1]}) = {value}")
